In [2]:
from matplotlib import pyplot as plt
from L500analysis.plotting.tools.figure_formatting import MyLogFormatter
%matplotlib inline

We first plot the pressure profiles from the stacked McDonald+14 Chandra data at $<z>=0.4$ and $<z>=0.8$.

In [3]:
import numpy as np
from SZmaps.utils.obs_utils import general_nfw_profile, McDonald14_params

rbins = np.arange(.1,5.,0.01)
def plot_McDonald(rbins) :
    plt.plot(rbins, general_nfw_profile(rbins,**McDonald14_params['z=0.4']),ls=':',label='M14: z=0.4')
    plt.plot(rbins, general_nfw_profile(rbins,**McDonald14_params['z=0.8']),ls=':',label='M14: z=0.8')
    plt.xscale('log')
    plt.yscale('log')
    plt.ylabel('$\\tilde{P}$',fontsize='xx-large')
    plt.xlabel('r/R$_{500c}$',fontsize='xx-large')
    plt.legend()
    plt.gca().xaxis.set_major_formatter(MyLogFormatter())
    plt.gca().yaxis.set_major_formatter(MyLogFormatter())

Now we compare with the L500 NR, CSF, and AGN average pressure profiles with different averaged weighting

In [4]:
import numpy as np

def calculate_scaled_profile(aexp, r_mid, r500c, profile, M500c, rbins=rbins)  :
    radii_scaled_by_500c = r_mid / r500c
    pressure_scaled_by_500c = calculate_scaled_pressure(profile, aexp, M500c)
    profile_scaled_by_500c = np.interp(rbins, 
                                       radii_scaled_by_500c, 
                                       pressure_scaled_by_500c)
    assert(profile_scaled_by_500c.shape == rbins.shape)
    return profile_scaled_by_500c
    
def calculate_scaled_pressure(pressure_profile, aexp, M500c) :
    return pressure_profile / calculate_P500(M500c,aexp)

def calculate_P500(M500c, aexp) :
    from astropy.cosmology import WMAP9 as cosmo
    '''Returns P500 in erg/cm**3. M500c expected in Msun/h'''
    return 1.45*1e-11 * (M500c / 1e15)**(2./3) * cosmo.efunc(1./float(aexp) - 1.)

def test_calculate_P500() :
    aexp = 1.
    m500c = 1e15
    assert(calculate_P500(1e15, aexp) == 1.45*1e-11)

def test_calculate_scaled_pressure() :
    import numpy as np
    flat_pressure_profile = np.zeros(100)+1.
    aexp =1.
    #print calculate_scaled_pressure(flat_pressure_profile, aexp, 1e15)
    assert(np.all(calculate_scaled_pressure(flat_pressure_profile, aexp, 1e15) == 1/(1.45*1e-11)))

def test_calculate_scaled_profile() :
    import numpy as np
    aexp = 1.
    r_mid = np.arange(0.0, 10,.1)
    r500c = 500.
    profile = np.zeros(100)+1.
    M500c = 1e15
    assert(np.all(calculate_scaled_profile(aexp,r_mid, r500c, profile, M500c,rbins=rbins[:len(r_mid)])\
                  == np.zeros(100)+1./(1.45*1e-11) ) )
    
test_calculate_P500()
test_calculate_scaled_pressure()
test_calculate_scaled_profile()

# Normalization function collections
def pressure_critical_density_evolution(aexp) :
    '''Returns E(z)**(2/3)'''
    from astropy.cosmology import WMAP9 as cosmo
    return cosmo.efunc( 1./float(aexp) - 1. )

def calculate_average_normalized_profiles(halo_profiles_dict, halo_properties_dict, halo_ids, profile_name, evolution_normalization_function=None, M500min=1e13) :
    '''evolution_normalization_function should be from normalization_collections'''
    from collections import defaultdict
    average_profiles = defaultdict(list)
    for aexp in halo_profiles_dict.keys() :
        halo_profiles_dataframe = halo_profiles_dict[aexp]
        halo_properties_dataframe = halo_properties_dict[aexp]
        for halo_id in halo_ids[aexp] :
            halo_id_profile_dataframe = halo_profiles_dataframe[halo_profiles_dataframe['id']==halo_id]
            halo_id_property_dataframe = halo_properties_dataframe[halo_properties_dataframe['id']==halo_id]
            r_mid = halo_id_profile_dataframe.as_matrix(columns=['r_mid'])[:,0]
            profile = halo_id_profile_dataframe.as_matrix(columns=[profile_name])[:,0]
            r500c = halo_id_property_dataframe.as_matrix(columns=['r500c'])[:,0]
            M500c = halo_id_property_dataframe.as_matrix(columns=['M_total_500c'])[:,0]
            if M500c < M500min :
                continue
            average_profiles[aexp].append(calculate_scaled_profile(aexp, r_mid, r500c, profile, M500c, rbins=rbins))
        
        average_profiles[aexp] = np.array(average_profiles[aexp]).mean(axis=0)
    return average_profiles

In [5]:
# Refactoring the functions in the cell above so they can apply to any thermodynamic profile 
# scaling (critical, mean, etc.) and averaging (mean, median, etc.)

def calculate_rbinned_scaled_profile(profile_scaling_function, profile, r_mid, rnormalization, 
                             rbins=rbins, aexp=None, mass=None, **profile_scaling_kwargs) :
                             
    ''' Note: Typically, profile_scaling_kwargs will need aexp and mass (assuming 
    that scaling is mass and aexp dependent).  
    Also, assumes that rbins has been defined in same level module'''

    scaled_radii = r_mid / rnormalization
    scaled_profile = calculate_scaled_profile(profile_scaling_function, profile,
                                             **profile_scaling_kwargs)
    
    scaled_profile_in_rbins = np.interp(rbins, scaled_radii, scaled_profile) 
    
    assert(scaled_profile_in_rbins.shape == rbins.shape)
    
    return scaled_profile_in_rbins

def calculate_scaled_profile(profile_scaling_function, profile, 
                             **profile_scaling_kwargs) :
    '''Returns profile scaled by some function'''
    return profile / profile_scaling_function(**profile_scaling_kwargs)


###  NEED TO MOVE THE FOLLOWING TO scaling_function_collections.py ####

def pressure_critical_scaling(aexp=None, mass=None) :
    '''Calculates P_deltac.  Part of profile_scaling_function_collections. 
    Returns P_deltac in erg/cm**3. M_deltac (mass) expected in Msun/h'''
    
    from astropy.cosmology import WMAP9 as cosmo
    return 1.45*1e-11 * (mass / 1e15)**(2./3) * cosmo.efunc(1./float(aexp) - 1.)**(8./3)

def pressure_mean_scaling(aexp=None, mass=None) :
    '''Calculates P_deltam.  Part of profile_scaling_function_collections. 
    Returns P_deltam in erg/cm**3.  M_deltam (mass) expected in Msun/h'''
    
    return 1.45*1e-11 * (mass/1e15)**(2./3) * (1./aexp)**(8./3)

def temperature_critical_scaling(aexp=None, mass=None) :
    '''Calculates T_deltac.  Part of profile_scaling_function_collections. 
    Returns T_deltac in keV. M_deltac (mass) expected in Msun/h'''
    from astropy.cosmology import WMAP9 as cosmo    
    return 11.05 * (mass/1e15)**(2./3) * cosmo.efunc(1./float(aexp)-1.)**(2./3)

def temperature_mean_scaling(aexp=None, mass=None) :
    '''Calculates T_deltam.  Part of profile_scaling_function_collections. 
    Returns T_deltam in keV. M_deltam (mass) expected in Msun/h'''
    from astropy.cosmology import WMAP9 as cosmo    
    return 11.05 * (mass/1e15)**(2./3) * (1./float(aexp))**(2./3)

def entropy_critical_scaling(aexp=None, mass=None) :
    '''Calculates K_deltac.  Part of profile_scaling_function_collections. 
    Returns K_deltac in keV/cm**2. M_deltac (mass) expected in Msun/h'''
    from astropy.cosmology import WMAP9 as cosmo    
    return 1963 * (mass/1e15)**(2./3) * cosmo.efunc(1./float(aexp)-1.)**(-2./3)

def entropy_mean_scaling(aexp=None, mass=None) :
    '''Calculates K_deltam.  Part of profile_scaling_function_collections. 
    Returns K_deltam in keV/cm**2. M_deltam (mass) expected in Msun/h'''
    from astropy.cosmology import WMAP9 as cosmo    
    return 1963 * (mass/1e15)**(2./3) * (1./aexp)**(-2./3)

########################################################################################

def calculate_scaled_profile_stats(halo_profiles_dict, halo_properties_dict, halo_ids, aexp, 
                                    profile_name=None, profile_scaling_function=None, 
                                   halo_props_kwargs=None,
                                    mass_min=1e13) :
    '''Get all the scaled profiles, calculate mean, median, and/or other quantities'''

    scaled_profiles = []
    halo_profiles_dataframe = halo_profiles_dict[aexp]
    halo_properties_dataframe = halo_properties_dict[aexp]
    for halo_id in halo_ids :
        halo_id_profile_dataframe = halo_profiles_dataframe[halo_profiles_dataframe['id']==halo_id]
        halo_id_property_dataframe = halo_properties_dataframe[halo_properties_dataframe['id']==halo_id]
        r_mid = halo_id_profile_dataframe.as_matrix(columns=['r_mid'])[:,0]
        profile = halo_id_profile_dataframe.as_matrix(columns=[profile_name])[:,0]
        rnormalization = halo_id_property_dataframe.as_matrix(columns=['r500c'])[:,0]
        mass = halo_id_property_dataframe.as_matrix(columns=['M_total_500c'])[:,0]
        if mass < mass_min :
            continue
        scaled_profile = calculate_rbinned_scaled_profile(profile_scaling_function, 
                                                          profile, r_mid, rnormalization, 
                                                        rbins=rbins, aexp=aexp, 
                                                         **profile_scaling_kwargs)
        scaled_profiles.append(scaled_profile)
        
    scaled_profiles = np.array(scaled_profiles)
    
    return scaled_profiles.mean(axis=0), scaled_profiles.median(axis=0), \
            np.std(scaled_profiles, axis=0)


In [6]:
# Get the closest aexps corresponding to average redshifts of the McDonald+14 sample
aexps = [1./(1. + redshift) for redshift in [0.4, 0.8]]

def get_simulation_average_profiles(database_name='L500_NR_0', database_dir='/home/babyostrich/data/databases/',
                                  rscales=['500c'], aexps=aexps, profiles=['P_mw','P_vw', 'Bulk_P_vw', 'Bulk_P_mw'],
                                   M500min=3e14) :
    import L500analysis.caps.io.reader as db

    # Load the simulation
    sim = db.Simulation(database_name,db_dir=database_dir)
    sim_aexps = [sim.find_aexp(aexp) for aexp in aexps]
    
    # Get the halo_ids
    halo_ids = {aexp: sim.get_halo_ids(aexp) for aexp in sim_aexps}
    
    # Get necessary properties: 'r500c, r200m, M_total_500c, M_total_200m' from halos table; 'P_mw, P_vw, Bulk_P_vw, Bulk_P_mw'
    props = ['r'+r for r in rscales] + ['M_total_'+r for r in rscales]
    profiles = ['P_mw', 'P_vw', 'Bulk_P_vw', 'Bulk_P_mw']

    # Select the halo properties and profiles for the aexp
    halo_properties_dict = {aexp: sim.get_halo_properties(halo_ids[aexp], props, aexp) 
                               for aexp in sim_aexps}
    halo_profiles_dict = {aexp: sim.get_halo_profiles(halo_ids[aexp], profiles, aexp) 
                               for aexp in sim_aexps}    
    
    average_profiles = {}
    for profile_name in profiles :  
        average_profiles[profile_name] = calculate_average_normalized_profiles(halo_profiles_dict, 
                                                                               halo_properties_dict, 
                                                                               halo_ids, profile_name, M500min=M500min)
    
    return average_profiles

In [7]:
M500min=3e14
nr_avg_profiles = get_simulation_average_profiles(database_name='L500_NR_0',M500min=M500min)
csf_avg_profiles = get_simulation_average_profiles(database_name='L500_CSF_0',M500min=M500min)
agn_avg_profiles = get_simulation_average_profiles(database_name='L500_AGN_0',M500min=M500min)


looking in /home/babyostrich/data/databases/ for L500_NR_0.db


TypeError: calculate_scaled_profile() takes exactly 2 arguments (6 given)

Plotting some simulation profiles.  Overall, what we see is that the simulations have higher normalization than the McDonald+14 profiles.  However, there is an analogous trend with redshift.  The high redshift bin has a higher normalized pressure in the inner regions.  As the scaled radius increase, the profiles cross over, and the high redshift bin has lower pressure. Note: this trend does not hold if we allow low mass halos in the sample.

In [ ]:
def plot_sim_pressure_w_McDonald(avg_profiles, Pkey=None) :
    for aexp in avg_profiles[Pkey].keys() :
        plt.plot(rbins, avg_profiles[Pkey][aexp],ls='-.',label=Pkey+': z=%.1f'%(1./aexp-1.))
    plot_McDonald(rbins)    

In [ ]:
# Velocity weighted pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'P_vw')

In [ ]:
# Mass weighted pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'P_mw')

In [ ]:
# Mass weighted bulk pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'Bulk_P_mw')

In [ ]:
# Velocity weighted pressure
plot_sim_pressure_w_McDonald(csf_avg_profiles, Pkey = 'P_vw')

In [ ]:
# Mass weighted pressure
plot_sim_pressure_w_McDonald(csf_avg_profiles, Pkey = 'P_mw')

In [ ]:
# Mass weighted bulk pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'Bulk_P_mw')

In [ ]:
# Velocity weighted pressure
plot_sim_pressure_w_McDonald(agn_avg_profiles, Pkey = 'P_vw')

In [ ]:
# Mass weighted pressure
plot_sim_pressure_w_McDonald(agn_avg_profiles, Pkey = 'P_mw')

In [ ]:
# Mass weighted bulk pressure
plot_sim_pressure_w_McDonald(agn_avg_profiles, Pkey = 'Bulk_P_mw')

The same allowing low mass halos into the sample.

In [ ]:
M500min=1e13
nr_avg_profiles = get_simulation_average_profiles(database_name='L500_NR_0',M500min=M500min)
csf_avg_profiles = get_simulation_average_profiles(database_name='L500_CSF_0',M500min=M500min)
agn_avg_profiles = get_simulation_average_profiles(database_name='L500_AGN_0',M500min=M500min)


In [ ]:
# Velocity weighted pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'P_vw')

In [ ]:
# Mass weighted pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'P_mw')

In [ ]:
# Mass weighted bulk pressure
plot_sim_pressure_w_McDonald(nr_avg_profiles, Pkey = 'Bulk_P_mw')